In [0]:
df = spark.read.option("header","true").option("inferSchema","true").csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

In [0]:
df.printSchema()

In [0]:
display(df)

In [0]:
from pyspark.sql import functions as F

top_5_products = (
    df.filter(F.col("event_type")=="purchase")
      .groupBy("product_id","brand")
      .agg(F.sum("price").alias("revenue"))
      .orderBy(F.col("revenue").desc())
      .limit(5)
)

display(top_5_products)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

window_spec = Window.partitionBy(F.col("user_id")).orderBy("event_time")

total_per_user = (df
                  .withColumn("cumulative_count",F.count("*").over(window_spec))
                  )
display(total_per_user)

In [0]:
s = df.select("event_time","user_id").filter("user_id=278272605")
display(s)

In [0]:
from pyspark.sql import functions as F

conversion = (
    df
    .groupBy("category_code")
    .pivot("event_type", ["view", "purchase"])
    .count()
    .withColumn(
        "conversion_rate",
        (F.col("purchase") / F.col("view")) * 100
    )
)

display(conversion)
